In [ ]:
import os
import sys
sys.path.append('../')

from settings import config as cfg
from utils import io_utils as iou
from utils import waveform_plot_utils as wpu
from utils import fourier_math_utils as fmu
from settings import period_bounds as pb
from utils import general_display_utils as gdu

# Prepare directory structure

In [ ]:
iou.create_directory_structure()

# Loading sounds

In [ ]:
files = [os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, name) for name in os.listdir(cfg.PATH_INSTRUMENT_SAMPLES)]
files.sort(key=lambda x: x.lower()) # making sure the order is the same as in period_bounds.py config file
sounds = []

for file in files:
    path = os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, file)
    sound, rate = iou.load_sound(path)
    sounds.append((sound, rate))

# Playing sounds

In [ ]:
iou.play_audio(files, n_columns=4)

# Plotting waveforms

In [ ]:
wpu.plot_waveform(sounds, files)

# Extracting period

In [ ]:
periods = []
rates = []

one_period_signals, sample_rates = fmu.extract_periods_and_data_rates(sounds)
one_period_audios = iou.export_and_store_one_period_audio(files, one_period_signals, sample_rates)

wpu.plot_waveform(sounds, files, mark_one_period = True)

# Calculating Fourier coefficients

In [ ]:
fourier_coefficients_per_instrument = []

for one_period_signal, N in zip(one_period_signals, cfg.N_HARMONICS_PER_INSTRUMENT):
    fourier_coefficients_per_instrument.append(fmu.calculate_fourier_coefficients(one_period_signal, N))

# Representing signal as mathematical function

In [ ]:
mathematical_representation_of_signal_per_instrument = []

for fourier_coefficients, period_bounds in zip(fourier_coefficients_per_instrument, pb.PERIOD_BOUNDS.values()):
    T = period_bounds[1] - period_bounds[0]
    mathematical_representation_of_signal_per_instrument.append(fmu.get_mathematical_representation_of_signal(fourier_coefficients, T))

gdu.print_mathematical_representation_of_signal(files, mathematical_representation_of_signal_per_instrument)

# Reconstruct original signal

In [ ]:
reconstructed_signals = []

for one_period_signal, fourier_coefficients in zip(one_period_signals, fourier_coefficients_per_instrument):
    reconstructed_signals.append(fmu.reconstruct_original_signal(one_period_signal, fourier_coefficients))
    
gdu.display_reconstructed_and_original_audio(files, reconstructed_signals, one_period_signals, sample_rates)

# Drawing power spectra of harmonics

In [ ]:
relative_harmonic_powers_per_instrument = []

for fourier_coefficients in fourier_coefficients_per_instrument:
    relative_harmonic_powers_per_instrument.append(fmu.calculate_harmonic_power_spectrum(fourier_coefficients))
    
gdu.draw_harmonics_power_spectra(files, relative_harmonic_powers_per_instrument)

# Plotting individual harmonics

In [ ]:
gdu.plot_individual_harmonics(
    files, mathematical_representation_of_signal_per_instrument
)

# Show reconstruction timeline

In [ ]:
wpu.draw_waveform_reconstruction_timeline(files, one_period_signals, sample_rates)

# Save sound of each harmonic

In [ ]:
iou.save_harmonics_sounds(files, mathematical_representation_of_signal_per_instrument, relative_harmonic_powers_per_instrument)